# Yelp Data Mining Project
## CMPE - 255 Data Mining Fall 2017
### Group 6
- Dhrumil Shah
- Nishant Rathi
- Rashmi Sharma

## Notebook to Preprocess Data and create CSR

In this Jupyter Notebook, we are loading data from yelp data set sql dump. We have considered data using following constraints:
- Users who have written 50 or more reviews
- Their reviews
- Rating of each of the review

This data is grouped into 80% Training data and 20% Test data.
We have grouped data by user ID and then split it.

In [4]:
import random
import numpy as np
import pandas as pd

In [5]:
def test_train_split_per_groupby(review_data,groupby, testperc):
    uids=review_data[groupby].unique()
    full_size = len(uids)
    choose = (int)(((float)(testperc)/100) * full_size)
    indices = np.random.choice(len(uids), choose, replace=False)
    full_indices = [m for m in range(0,len(uids))]
    train_indices = set(full_indices)-set(indices)
    grouped =review_data.groupby([groupby])
    test_indices_final=[]  
    train_indices_final = []
    for g in grouped.groups:
        full_size = len(grouped.groups[g])
        choose = (int)(((float)(testperc)/100) * full_size)
        indices = np.random.choice(full_size, choose, replace=False)
        full_indices = [m for m in range(0,full_size)]
        train_indices = set(full_indices)-set(indices)
        train_indices = list(train_indices)
        test_indices_final.extend(list(grouped.groups[g][indices]))
        train_indices_final.extend(list(grouped.groups[g][train_indices]))
    
    
    saveGroupedData("data/test_"+str(groupby)+".dat",test_indices_final)
    saveGroupedData("data/train_"+str(groupby)+".dat",train_indices_final)

def saveGroupedData(filename, indices):
    output_file = open(filename, 'w')
    for i in indices:
        output_file.write(str(i)+"\n")
    output_file.close()    

In [ ]:
review_data_all = pd.read_csv('dt.dat',"\t")
test_train_split_per_groupby(review_data_all,'UID',80)
test_train_split_per_groupby(review_data_all,'BusinessId',20)

In [5]:
review_data_all = pd.read_csv('data/reviews_with_counter.dat',"\t")

### Taking only 100K data

In [6]:
review_data = review_data_all[:100000]

In [7]:
test_train_split (review_data,20)

In [8]:
with open("data/reviews.dat", "r") as fh:
    lines = fh.readlines() 

In [9]:
with open("data/test_temp.dat", "r") as fh:
    test_idx = fh.readlines() 

In [10]:
with open("data/train_temp.dat", "r") as fh:
    train_idx = fh.readlines() 

In [11]:
train_file = open("data/train_final.dat", 'w')
for i in train_idx:
    train_file.write(lines[int(i)-1])
train_file.close()

In [12]:
test_file = open("data/test_final.dat", 'w')
for i in test_idx:
    test_file.write(lines[int(i)-1])
test_file.close()